In [233]:
import numpy as np
import pandas as pd

In [234]:
BASE = "./data/"
train_data = pd.read_csv(BASE + "train.csv")
test_data = pd.read_csv(BASE + "test.csv")
sub_data = pd.read_csv(BASE + "sample_submission.csv")

In [235]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [236]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [237]:
sub_data.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [238]:
tweets = train[['text', 'target']]

In [239]:
tweets.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [240]:
tweets.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [241]:
tweets.shape

(7613, 2)

In [242]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import string
stemmer = SnowballStemmer("english")
stop_words = stopwords.words('english')

In [243]:

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def stopwords(text):
    text =  [word.lower() for word in text.split() if word.lower() not in stop_words]
    return " ".join(text) 

def stemming(text):    
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text) 

In [244]:
tweets['text'] = tweets['text'].apply(remove_punctuation)
tweets['text'] = tweets['text'].apply(stopwords)
tweets['text'] = tweets['text'].apply(stemming)

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [245]:
vectorizer = CountVectorizer(analyzer='word', binary=True)
vectorizer.fit(tweets['text'])

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [246]:
X = vectorizer.transform(tweets['text']).todense()
y = tweets['target'].values
X.shape, y.shape

((7613, 19319), (7613,))

In [247]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [248]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 145)

In [249]:
model = LogisticRegression()
model.fit(X_train, y_train)

F:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [250]:
y_pred = model.predict(X_test)

f1score = f1_score(y_test, y_pred)
print(f"Model Score: {f1score * 100} %")

Model Score: 74.06143344709896 %


In [251]:
tweets_test = test['text']
test_X = vectorizer.transform(tweets_test).todense()
test_X.shape

(3263, 19319)

In [252]:
lr_pred = model.predict(test_X)

In [253]:
sub['target'] = lr_pred
sub.to_csv("submission.csv", index=False)
sub.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1
